In [ ]:
debugMode = False

import numpy as np
# from numba import jit, prange
import numba as nb
import numba as nb
import scipy
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from scipy.fft import fft, ifft, fft2, ifft2
from scipy.fft import dst,  idst, fft, ifft, fftshift, ifftshift, fftfreq
from scipy.linalg import polar

from math import sqrt, tanh, ceil, floor, log10, exp, cos, sin, copysign

pi = np.pi
π = np.pi

%run plotting.ipynb
%run eulerianGrid.ipynb
%run lagrangianParticles.ipynb
%run levelSet.ipynb


    



# serial version

In [ ]:
@nb.jit(nopython=True, fastmath=True)  
def N(x):
    d = abs(x)
    if d < 1.:
        return (.5*d - 1.)*d*d + 2./3.
    elif d < 2.:
        return ((-1./6.*d + 1)*d-2.)*d + 4./3.
    else:
        return 0.
    
@nb.jit(nopython=True, fastmath=True) 
def Nd(x):
    d = abs(x)
    if d < 1.:
        return (1.5*d - 2.)*x
    elif d < 2.:
        return copysign((-.5*d + 2.)*d - 2., -x)
    else:
        return 0.

## determinant of 3x3 matrix
@nb.jit(nopython=True, fastmath=True)
def dete(a):
    return (a[0,0] * (a[1,1] * a[2,2] - a[2,1] * a[1,2])
           -a[1,0] * (a[0,1] * a[2,2] - a[2,1] * a[0,2])
           +a[2,0] * (a[0,1] * a[1,2] - a[1,1] * a[0,2]))


@nb.jit(nopython=True, parallel=True, fastmath=True, cache=True)
def substep1(X, U, M,
             xx, yy, zz, mm, uu, vv, ww, hx, hy, hz, 
             ib, jb, kb, it, jt, kt):
    for n in range(M.shape[0]):
        for i in nb.prange(ib[n], it[n]):
            w0 = N( ( X[n,0]-xx[i,0,0] ) / hx )
            for j in range(jb[n], jt[n]):
                w1 = N( ( X[n,1]-yy[0,j,0] ) / hy )
                for k in range(kb[n], kt[n]):
                    w = w0 * w1 * N( ( X[n,2]-zz[0,0,k] ) / hz ) 
                    w *= M[n, 0]
                    mm[i,j,k] += w  
                    uu[i,j,k] += w * U[n,0]
                    vv[i,j,k] += w * U[n,1]
                    ww[i,j,k] += w * U[n,2]
                    
def step1(eg, lp):
    eg.mm[:] = 0.
    eg.uu[:] = 0.
    eg.vv[:] = 0.
    eg.ww[:] = 0.

    ic = np.floor( (lp.X[:,0]-eg.xx[0,0,0])/eg.hx ).astype(int)
    jc = np.floor( (lp.X[:,1]-eg.yy[0,0,0])/eg.hy ).astype(int)
    kc = np.floor( (lp.X[:,2]-eg.zz[0,0,0])/eg.hz ).astype(int)

    ib = np.maximum(0, (ic[:]-2) % Nx)
    jb = np.maximum(0, (jc[:]-2) % Ny)
    kb = np.maximum(0, (kc[:]-2) % Nz)

    it = np.minimum(Nx-1, (ic[:]+3) % Nx)
    jt = np.minimum(Ny-1, (jc[:]+3) % Ny)
    kt = np.minimum(Nz-1, (kc[:]+3) % Nz)

    substep1(lp.X, lp.U, lp.M,
             eg.xx, eg.yy, eg.zz, eg.mm, eg.uu, eg.vv, eg.ww, eg.hx, eg.hy, eg.hz,
             ib, jb, kb, it, jt, kt)
    
    ## normalize velocities by eulerian mass
    eg.uu[:] = eg.uu/(eg.mm+1e-12)
    eg.vv[:] = eg.vv/(eg.mm+1e-12)  
    eg.ww[:] = eg.ww/(eg.mm+1e-12)  
    

    
@nb.jit(nopython=True, parallel=True, fastmath=True, cache=True)
def substep2(X, U, M,
             xx, yy, zz, mm, uu, vv, ww, hx, hy, hz, 
             ib, jb, kb, it, jt, kt):
    
    gridCellVol = hx*hy*hz
    for n in nb.prange(M.shape[0]):
        M[n,1:] = 0.
        for i in range(ib[n], it[n]):
            w0 = N( ( X[n,0]-xx[i,0,0] ) / hx )
            for j in range(jb[n], jt[n]):
                w1 = N( ( X[n,1]-yy[0,j,0] ) / hy )
                for k in range(kb[n], kt[n]):
                    w = w0 * w1 * N( ( X[n,2]-zz[0,0,k] ) / hz ) 
                    M[n,1] += w * mm[i,j,k]
        M[n,1] = M[n,1] / gridCellVol
        M[n,2] = M[n,0] / M[n,1]    

def step2(eg, lp):
    ic = np.floor( (lp.X[:,0]-eg.xx[0,0,0])/eg.hx ).astype(int)
    jc = np.floor( (lp.X[:,1]-eg.yy[0,0,0])/eg.hy ).astype(int)
    kc = np.floor( (lp.X[:,2]-eg.zz[0,0,0])/eg.hz ).astype(int)

    ib = np.maximum(0, (ic[:]-2) % Nx)
    jb = np.maximum(0, (jc[:]-2) % Ny)
    kb = np.maximum(0, (kc[:]-2) % Nz)

    it = np.minimum(Nx-1, (ic[:]+3) % Nx)
    jt = np.minimum(Ny-1, (jc[:]+3) % Ny)
    kt = np.minimum(Nz-1, (kc[:]+3) % Nz)
    
    substep2(lp.X, lp.U, lp.M,
             eg.xx, eg.yy, eg.zz, eg.mm, eg.uu, eg.vv, eg.ww, eg.hx, eg.hy, eg.hz, 
             ib, jb, kb, it, jt, kt)   

def step3(eg, lp):
    eg.ffx[:] = 0.
    eg.ffy[:] = 0.
    eg.ffz[:] = 0.
    
    ic = np.floor( (lp.X[:,0]-eg.xx[0,0,0])/eg.hx ).astype(int)
    jc = np.floor( (lp.X[:,1]-eg.yy[0,0,0])/eg.hy ).astype(int)
    kc = np.floor( (lp.X[:,2]-eg.zz[0,0,0])/eg.hz ).astype(int)

    ib = np.maximum(0, (ic[:]-2) % Nx)
    jb = np.maximum(0, (jc[:]-2) % Ny)
    kb = np.maximum(0, (kc[:]-2) % Nz)

    it = np.minimum(Nx-1, (ic[:]+3) % Nx)
    jt = np.minimum(Ny-1, (jc[:]+3) % Ny)
    kt = np.minimum(Nz-1, (kc[:]+3) % Nz)
    
    lp.svdu[:], lp.svds[:], lp.svdv[:] = np.linalg.svd(lp.Fe)
    
    substep3(lp.X, lp.U, lp.M, lp.Fe, lp.Fp, lp.lameParam, lp.Ψ, lp.σ, lp.svdu, lp.svdv,
             eg.xx, eg.yy, eg.zz, eg.mm, eg.uu, eg.vv, eg.ww, eg.hx, eg.hy, eg.hz, 
             eg.ffx, eg.ffy, eg.ffz,
             ib, jb, kb, it, jt, kt)
    
    
@nb.jit(nopython=True, parallel=True, fastmath=True, cache=True)
def substep3(X, U, M, Fe, Fp, lameParam, Ψ, σ, svdu, svdv,
             xx, yy, zz, mm, uu, vv, ww, hx, hy, hz, 
             ffx, ffy, ffz,
             ib, jb, kb, it, jt, kt):
    
    I = np.eye(3)
    for n in range(M.shape[0]):
        Re = svdu[n,:,:] @ svdv[n,:,:] 
        Je = dete(Fe[n,:,:])
        Jp = dete(Fp[n,:,:])
        lameParam[n,0] = μ0*exp(ξ*(1-Jp))
        lameParam[n,1] = λ0*exp(ξ*(1-Jp))
        FNS = np.linalg.norm(Fe[n,:,:]-Re)**2 ## frobenius nnorm squared
        Ψ[n] = lameParam[n,0] * FNS + .5*lameParam[n,1] * (Je-1.)**2
        c1 = 2*lameParam[n,0]##/Jp
        c2 = lameParam[n,1] * (Je-1.) * Je ##/Jp
        σ[n,:,:] = c1 * ( Fe[n,:,:] - Re) @ (Fe[n,:,:].T ) + c2 * I
        
        for i in nb.prange(ib[n], it[n]):
            w0 = N(  ( X[n,0]-xx[i,0,0] ) / hx )
            wx = Nd( ( X[n,0]-xx[i,0,0] ) / hx ) / hx
            for j in range(jb[n], jt[n]):
                w1 = N( (  X[n,1]-yy[0,j,0] ) / hy )
                wy = Nd( ( X[n,1]-yy[0,j,0] ) / hy ) / hy
                for k in range(kb[n], kt[n]):
                    w2   = N(  ( X[n,2]-zz[0,0,k] ) / hz ) 
                    wz   = Nd( ( X[n,2]-zz[0,0,k] ) / hz ) / hz
                    
                    wip  = w0 * w1 * w2
                    wipx = wx * w1 * w2
                    wipy = w0 * wy * w2
                    wipz = w0 * w1 * wz
                    
                    ffx[i,j,k] += -M[n,2] * ( σ[n,0,0] * wipx 
                                            + σ[n,0,1] * wipy
                                            + σ[n,0,2] * wipz )
                    ffy[i,j,k] += -M[n,2] * ( σ[n,1,0] * wipx 
                                            + σ[n,1,1] * wipy
                                            + σ[n,1,2] * wipz ) \
                                  -M[n,0] * g * wip  
                    ffz[i,j,k] += -M[n,2] * ( σ[n,2,0] * wipx 
                                            + σ[n,2,1] * wipy
                                            + σ[n,2,2] * wipz )
                    

def step4(eg, lp):
    eg.uuo[:] = eg.uu
    eg.vvo[:] = eg.vv
    eg.wwo[:] = eg.ww
    
    eg.uu[:] = eg.uu + dt*eg.ffx/(eg.mm+1e-12)
    eg.vv[:] = eg.vv + dt*eg.ffy/(eg.mm+1e-12)
    eg.ww[:] = eg.ww + dt*eg.ffz/(eg.mm+1e-12)

    
def step5(eg, lp, ls):
    if ls.firstTime == True:
        eg.bb  = (ls.phi <= 0.)
        eg.bbb = ( ( eg.yy-2*eg.hy +1.) <= 1e-12) ## stop before hit floor
        ls.firstTime = False

    ## check vs box 
    eg.vn[eg.bb]  = eg.uu[eg.bb] * ls.normalx[eg.bb] \
                  + eg.vv[eg.bb] * ls.normaly[eg.bb]
    eg.uut[eg.bb] = eg.uu[eg.bb] - eg.vn[eg.bb] * ls.normalx[eg.bb]
    eg.vvt[eg.bb] = eg.vv[eg.bb] - eg.vn[eg.bb] * ls.normaly[eg.bb]
    eg.wwt[eg.bb] = eg.ww[eg.bb]
    eg.vt[eg.bb]  = np.sqrt( eg.uu[eg.bb] * eg.uu[eg.bb] 
                           + eg.vv[eg.bb] * eg.vv[eg.bb]
                           + eg.ww[eg.bb] * eg.ww[eg.bb] )
    eg.bbf[eg.bb] = ( eg.vt[eg.bb] <= -μ*eg.vn[eg.bb] )
    
    ## check vs floor
    eg.vn[eg.bbb]  = eg.vv[eg.bbb] 
    eg.uut[eg.bbb] = eg.uu[eg.bbb]
    eg.vvt[eg.bbb] = 0.
    eg.wwt[eg.bbb] = eg.ww[eg.bbb]
    eg.vt[eg.bbb]  = np.sqrt( eg.uu[eg.bbb] * eg.uu[eg.bbb] 
                            + eg.vv[eg.bbb] * eg.vv[eg.bbb]
                            + eg.ww[eg.bbb] * eg.ww[eg.bbb] )
    eg.bbf[eg.bbb] = ( eg.vt[eg.bbb] <= -μ*eg.vn[eg.bbb] )

    ## dynamic friction
    eg.uu[eg.bb]   = (1. + μ*eg.vn[eg.bb]/(eg.vt[eg.bb]+1e-12))*eg.uut[eg.bb]
    eg.vv[eg.bb]   = (1. + μ*eg.vn[eg.bb]/(eg.vt[eg.bb]+1e-12))*eg.vvt[eg.bb]
    eg.ww[eg.bb]   = (1. + μ*eg.vn[eg.bb]/(eg.vt[eg.bb]+1e-12))*eg.wwt[eg.bb]
    eg.uu[eg.bbb]  = (1. + μ*eg.vn[eg.bbb]/(eg.vt[eg.bbb]+1e-12))*eg.uut[eg.bbb]
    eg.vv[eg.bbb]  = (1. + μ*eg.vn[eg.bbb]/(eg.vt[eg.bbb]+1e-12))*eg.vvt[eg.bbb]
    eg.ww[eg.bbb]  = (1. + μ*eg.vn[eg.bbb]/(eg.vt[eg.bbb]+1e-12))*eg.wwt[eg.bbb]

    ## static friction
    eg.uu[eg.bbf]  = 0.
    eg.vv[eg.bbf]  = 0.
    eg.ww[eg.bbf]  = 0.
    
## update particle's deformtion gradient
def step7(eg, lp):
    
    ic = np.floor( (lp.X[:,0]-eg.xx[0,0,0])/eg.hx ).astype(int)
    jc = np.floor( (lp.X[:,1]-eg.yy[0,0,0])/eg.hy ).astype(int)
    kc = np.floor( (lp.X[:,2]-eg.zz[0,0,0])/eg.hz ).astype(int)

    ib = np.maximum(0, (ic[:]-2) % Nx)
    jb = np.maximum(0, (jc[:]-2) % Ny)
    kb = np.maximum(0, (kc[:]-2) % Nz)

    it = np.minimum(Nx-1, (ic[:]+3) % Nx)
    jt = np.minimum(Ny-1, (jc[:]+3) % Ny)
    kt = np.minimum(Nz-1, (kc[:]+3) % Nz)
    
    substep7(lp.X, lp.M, 
             lp.Fu, lp.F, lp.Fe, lp.Fp, lp.Feh, lp.Fph,
             eg.xx, eg.yy, eg.zz, eg.mm, eg.uu, eg.vv, eg.ww, eg.hx, eg.hy, eg.hz, 
             eg.ffx, eg.ffy, eg.ffz,
             ib, jb, kb, it, jt, kt)
    
    ## compute svd then clamp sing. vals.
    lp.svdu[:], lp.svds[:], lp.svdv[:] = np.linalg.svd(lp.Feh)
    lp.svds[:] = np.clip(lp.svds[:], 1.-θc, 1.+θs)
    
    substep7_2(lp.X, lp.U, lp.M, lp.svdu, lp.svdv, lp.svds,
             lp.Fu, lp.F, lp.Fe, lp.Fp, 
             ib, jb, kb, it, jt, kt)

@nb.jit(nopython=True, parallel=True, fastmath=True, cache=True)
def substep7(X, M, 
             Fu, F, Fe, Fp, Feh, Fph,
             xx, yy, zz, mm, uu, vv, ww, hx, hy, hz, 
             ffx, ffy, ffz,
             ib, jb, kb, it, jt, kt):
    for n in nb.prange(M.shape[0]):
        Fu[n,0,0] = 1. 
        Fu[n,0,1] = 0.
        Fu[n,0,2] = 0.
        
        Fu[n,1,0] = 0.     
        Fu[n,1,1] = 1.
        Fu[n,1,2] = 0.
        
        Fu[n,2,0] = 0.
        Fu[n,2,1] = 0.
        Fu[n,2,2] = 1. 
        
        for i in range(ib[n], it[n]):
            w0 = N( ( X[n,0]-xx[i,0,0] ) / hx )
            wx = Nd( ( X[n,0]-xx[i,0,0] ) / hx ) / hx
            for j in range(jb[n], jt[n]):
                w1 = N(  ( X[n,1]-yy[0,j,0] ) / hy )
                wy = Nd( ( X[n,1]-yy[0,j,0] ) / hy ) / hy
                for k in range(kb[n], kt[n]):
                    w2   = N(  ( X[n,2]-zz[0,0,k] ) / hz ) 
                    wz   = Nd( ( X[n,2]-zz[0,0,k] ) / hz ) / hz

                    wipx = wx * w1 * w2 * dt
                    wipy = w0 * wy * w2 * dt
                    wipz = w0 * w1 * wz * dt
                    
#                     Fu[n,0,0] += uu[i,j,k] * wipx
#                     Fu[n,0,1] += uu[i,j,k] * wipy
#                     Fu[n,0,2] += uu[i,j,k] * wipz

#                     Fu[n,1,0] += vv[i,j,k] * wipx  
#                     Fu[n,1,1] += vv[i,j,k] * wipy 
#                     Fu[n,1,2] += vv[i,j,k] * wipz  

#                     Fu[n,2,0] += ww[i,j,k] * wipx  
#                     Fu[n,2,1] += ww[i,j,k] * wipy
#                     Fu[n,2,2] += ww[i,j,k] * wipz
                    
                    
                    Fu[n,0,0] += uu[i,j,k] * wipx
                    Fu[n,0,1] += vv[i,j,k] * wipx
                    Fu[n,0,2] += ww[i,j,k] * wipx

                    Fu[n,1,0] += uu[i,j,k] * wipy  
                    Fu[n,1,1] += vv[i,j,k] * wipy 
                    Fu[n,1,2] += ww[i,j,k] * wipy  

                    Fu[n,2,0] += uu[i,j,k] * wipz  
                    Fu[n,2,1] += vv[i,j,k] * wipz
                    Fu[n,2,2] += ww[i,j,k] * wipz 
                    
        ## the new deformation gradient
        F[n,:,:]  = Fu[n,:,:] @ Fe[n,:,:] @ Fp[n,:,:]
        
        ## temporarily define
        Feh[n,:,:] = Fu[n,:,:] @ Fe[n,:,:]
        Fph[n,:,:] = Fp[n,:,:]

    

@nb.jit(nopython=True, parallel=True, fastmath=True, cache=True)
def substep7_2(X, U, M, svdu, svdv, svds,
             Fu, F, Fe, Fp, 
             ib, jb, kb, it, jt, kt):
    for n in nb.prange(M.shape[0]):
        ## the final elastic & plastic components are
        Fe[n,:,:] = svdu[n,:,:]     @ np.diag(svds[n,:])   @ svdv[n,:,:]
        Fp[n,:,:] = (svdv[n,:,:]).T @ np.diag(1./svds[n,:]) @ (svdu[n,:,:]).T @ F[n,:,:]



        
## update particle's velocity
def step8(eg, lp):
    ic = np.floor( (lp.X[:,0]-eg.xx[0,0,0])/eg.hx ).astype(int)
    jc = np.floor( (lp.X[:,1]-eg.yy[0,0,0])/eg.hy ).astype(int)
    kc = np.floor( (lp.X[:,2]-eg.zz[0,0,0])/eg.hz ).astype(int)

    ib = np.maximum(0, (ic[:]-2) % Nx)
    jb = np.maximum(0, (jc[:]-2) % Ny)
    kb = np.maximum(0, (kc[:]-2) % Nz)

    it = np.minimum(Nx-1, (ic[:]+3) % Nx)
    jt = np.minimum(Ny-1, (jc[:]+3) % Ny)
    kt = np.minimum(Nz-1, (kc[:]+3) % Nz)
    
    substep8(lp.X, lp.U, lp.M, lp.Upic, lp.Uflip,
             eg.xx, eg.yy, eg.zz, eg.mm, eg.uu, eg.vv, eg.ww, eg.hx, eg.hy, eg.hz, 
             eg.ffx, eg.ffy, eg.ffz, eg.uuo, eg.vvo, eg.wwo,
             ib, jb, kb, it, jt, kt)


##pppp    
@nb.jit(nopython=True, parallel=True, fastmath=True, cache=True)
def substep8(X, U, M, Upic, Uflip,
             xx, yy, zz, mm, uu, vv, ww, hx, hy, hz, 
             ffx, ffy, ffz, uuo, vvo, wwo,
             ib, jb, kb, it, jt, kt):
    for n in nb.prange(M.shape[0]):
        Upic[n,:]  = 0.
        Uflip[n,:] = U[n,:]
        
        for i in range(ib[n], it[n]):
            w0 = N( ( X[n,0]-xx[i,0,0] ) / hx )
            for j in range(jb[n], jt[n]):
                w1 = N( ( X[n,1]-yy[0,j,0] ) / hy )
                for k in range(kb[n], kt[n]):
                    wip = w0 * w1 * N( ( X[n,2]-zz[0,0,k] ) / hz ) 
                    
                    ## obtain PIC
                    Upic[n,0] += wip * uu[i,j,k]
                    Upic[n,1] += wip * vv[i,j,k]
                    Upic[n,2] += wip * ww[i,j,k]
                    
                    ## obtain FLIP 
                    Uflip[n,0] += wip * (uu[i,j,k]-uuo[i,j,k])
                    Uflip[n,1] += wip * (vv[i,j,k]-vvo[i,j,k])
                    Uflip[n,2] += wip * (ww[i,j,k]-wwo[i,j,k])
                    
        
        ## update velocity
        U[n,:] = (1.-α)*Upic[n,:] + α*Uflip[n,:]
        
        
## update particle's position
def step10(eg, lp):
    substep10(lp.X, lp.U)

@nb.jit(nopython=True, parallel=True, fastmath=True, cache=True)
def substep10(X, U):
    for n in nb.prange(X.shape[0]):
        X[n,:] = X[n,:] + dt*U[n,:]
    

# Simulation parameters

In [ ]:
# sim params
Mx = 4
My = 4
Mz = 1
Nx = 100
Ny = 100
Nz = 100
numberParticles = 500000//27//10
tf = 1.
dt = 0.001
α = .95

# case 1

In [ ]:
# %%prun -s cumulative

%run lagrangianParticles.ipynb

## parameters
g = 9.81
μ = 10 ## coeff of friction

# material properties
E0 = 1.4e5 ## initial young modulus
ρ0 = 4e2 ## initial density & currently unused

ν = .2 ## poisson ration
λ0 = E0*ν/(1.+ν)/(1.-2*ν)
μ0 = E0/(1.+ν)/2

ξ = 10 ## Hardening coefficient
θc = 2.5e-2 ## Critical compression
θs = 7.5e-3 ## Critical stretch






eg = eulerianGrid(Nx, Ny, Nz)
lp = lagrangianParticles(numberParticles)
ls = levelSet(Nx, Ny, Nz)

snaptimes = np.arange(0, tf+.1, 1./30.)
dt = 0.0005
tt = np.arange(0, tf+dt, dt)
tt = np.concatenate( [tt, snaptimes])
tt = np.unique(tt)

currentT = 0.


partitions = lp.sort3D(Mx, My, Mz)

for i, t in enumerate(tt[:-1]):
    print(i)
    dt = tt[i+1]-tt[i]
    if tt[i]+dt>tf:
        dt = tf-tt[i] 
    currentT += dt
    
    step1(eg, lp)
    if i == 0:
        print(np.mean(lp.M[:,1]))
        step2(eg, lp)
        print(np.mean(lp.M[:,1]))
    step3(eg, lp)
    step4(eg, lp)
    step5(eg, lp, ls)
    step7(eg, lp)
    step8(eg, lp)
    step10(eg, lp)
        
    
#     if tt[i] in snaptimes:
    if (i % 10) == 0:
        fig, ax = createPlot()
        for k, part in enumerate(partitions):
            plt.plot(lp.X[part,0], lp.X[part,1], shapes[ floor(k/5) ], c=colors[k%5], ms=2)
        v = np.linspace(0., 800, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], np.sum(eg.mm/eg.hx/eg.hy/eg.hz, axis=-1),
                        levels=v, extend='both')
        ax.contour( eg.xx[:,:,0], eg.yy[:,:,0], ls.phi[:,:,0], [0.],
                          colors='k' )
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case1/particles'+str(i)+'.png')
        plt.show()
        plt.close()
        
        fig, ax = createPlot()
        v = np.linspace(-1., 1, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], eg.uu[:,:,Nx//2],
                        levels=v, extend='both')
        ax.contour( eg.xx[:,:,0], eg.yy[:,:,0], ls.phi[:,:,0], [0.],
                          colors='k' )
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case1/U'+str(i)+'.png')
        plt.close()
        
        fig, ax = createPlot()
        v = np.linspace(-1., 1, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], eg.vv[:,:,Nx//2],
                        levels=v, extend='both')
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case1/V'+str(i)+'.png')
        plt.close()
        
        lp.saveTxt('case1/', i)
        
        





# Case 2

In [ ]:
# %%prun -s cumulative
## parameters
g = 9.81
μ = .1 ## coeff of friction

# material properties
E0 = 1.4e5 ## initial young modulus
ρ0 = 4e2 ## initial density & currently unused

ν = .2 ## poisson ration
λ0 = E0*ν/(1.+ν)/(1.-2*ν)
μ0 = E0/(1.+ν)/2

ξ = 10 ## Hardening coefficient
θc = 2.5e-2 ## Critical compression
θs = 5.0e-3 ## Critical stretch






eg = eulerianGrid(Nx, Ny, Nz)
lp = lagrangianParticles(numberParticles)
ls = levelSet(Nx, Ny, Nz)


snaptimes = np.arange(0, tf+.1, 1./30.)
dt = 0.001
tt = np.arange(0, tf+dt, dt)
tt = np.concatenate( [tt, snaptimes])
tt = np.unique(tt)

currentT = 0.


partitions = lp.sort3D(Mx, My, Mz)

for i, t in enumerate(tt[:-1]):
    print(tt[i])
    dt = tt[i+1]-tt[i]
    if tt[i]+dt>tf:
        dt = tf-tt[i] 
    currentT += dt
    
    step1(eg, lp)
    if i == 0:
        print(np.mean(lp.M[:,1]))
        step2(eg, lp)
        print(np.mean(lp.M[:,1]))
    step3(eg, lp)
    step4(eg, lp)
    step5(eg, lp, ls)
    step7(eg, lp)
    step8(eg, lp)
    step10(eg, lp)
        
    
    if tt[i] in snaptimes:
        fig, ax = createPlot()
        for k, part in enumerate(partitions):
            plt.plot(lp.X[part,0], lp.X[part,1], shapes[ floor(k/5) ], c=colors[k%5], ms=2)
        v = np.linspace(0., 800, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], np.sum(eg.mm/eg.hx/eg.hy/eg.hz, axis=-1),
                        levels=v, extend='both')
        ax.contour( eg.xx[:,:,0], eg.yy[:,:,0], ls.phi[:,:,0], [0.],
                          colors='k' )
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case2/particles'+str(i)+'.png')
        plt.show()
        plt.close()
        
        fig, ax = createPlot()
        v = np.linspace(-1., 1, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], eg.uu[:,:,Nx//2],
                        levels=v, extend='both')
        ax.contour( eg.xx[:,:,0], eg.yy[:,:,0], ls.phi[:,:,0], [0.],
                          colors='k' )
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case2/U'+str(i)+'.png')
        plt.close()
        
        fig, ax = createPlot()
        v = np.linspace(-1., 1, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], eg.vv[:,:,Nx//2],
                        levels=v, extend='both')
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case2/V'+str(i)+'.png')
        plt.close()
        
        lp.saveTxt('case2/', i)
        
        





# Case 3

In [ ]:
# %%prun -s cumulative
## parameters
g = 9.81
μ = .03 ## coeff of friction

# material properties
E0 = 1.4e5 ## initial young modulus
ρ0 = 4e2 ## initial density & currently unused

ν = .2 ## poisson ration
λ0 = E0*ν/(1.+ν)/(1.-2*ν)
μ0 = E0/(1.+ν)/2

ξ = 10 ## Hardening coefficient
θc = 1.9e-2 ## Critical compression
θs = 5.0e-3 ## Critical stretch






eg = eulerianGrid(Nx, Ny, Nz)
lp = lagrangianParticles(numberParticles)
ls = levelSet(Nx, Ny, Nz)

snaptimes = np.arange(0, tf+.1, 1./30.)
dt = 0.001
tt = np.arange(0, tf+dt, dt)
tt = np.concatenate( [tt, snaptimes])
tt = np.unique(tt)

currentT = 0.


partitions = lp.sort3D(Mx, My, Mz)

for i, t in enumerate(tt[:-1]):
    dt = tt[i+1]-tt[i]
    if tt[i]+dt>tf:
        dt = tf-tt[i] 
    currentT += dt
    
    step1(eg, lp)
    if i == 0:
        print(np.mean(lp.M[:,1]))
        step2(eg, lp)
        print(np.mean(lp.M[:,1]))
        print(np.mean(lp.M[:,2]))
    step3(eg, lp)
    step4(eg, lp)
    step5(eg, lp, ls)
    step7(eg, lp)
    step8(eg, lp)
    step10(eg, lp)
        
    
    if tt[i] in snaptimes:
        fig, ax = createPlot()
        for k, part in enumerate(partitions):
            plt.plot(lp.X[part,0], lp.X[part,1], shapes[ floor(k/5) ], c=colors[k%5], ms=2)
        v = np.linspace(0., 800, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], np.sum(eg.mm/eg.hx/eg.hy/eg.hz, axis=-1),
                        levels=v, extend='both')
        ax.contour( eg.xx[:,:,0], eg.yy[:,:,0], ls.phi[:,:,0], [0.],
                          colors='k' )
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case3/particles'+str(i)+'.png')
        plt.show()
        plt.close()
        
        fig, ax = createPlot()
        v = np.linspace(-1., 1, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], eg.uu[:,:,Nx//2],
                        levels=v, extend='both')
        ax.contour( eg.xx[:,:,0], eg.yy[:,:,0], ls.phi[:,:,0], [0.],
                          colors='k' )
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case3/U'+str(i)+'.png')
        plt.close()
        
        fig, ax = createPlot()
        v = np.linspace(-1., 1, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], eg.vv[:,:,Nx//2],
                        levels=v, extend='both')
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case3/V'+str(i)+'.png')
        plt.close()
        
        lp.saveTxt('case3/', i)
        
        





# Case 4

In [ ]:
# %%prun -s cumulative
## parameters
g = 9.81
μ = .03 ## coeff of friction

# material properties
E0 = 1.4e5 ## initial young modulus
ρ0 = 4e2 ## initial density & currently unused

ν = .2 ## poisson ration
λ0 = E0*ν/(1.+ν)/(1.-2*ν)
μ0 = E0/(1.+ν)/2

ξ = 10 ## Hardening coefficient
θc = 1.9e-2 ## Critical compression
θs = 7.5e-3 ## Critical stretch






eg = eulerianGrid(Nx, Ny, Nz)
lp = lagrangianParticles(numberParticles)
ls = levelSet(Nx, Ny, Nz)

snaptimes = np.arange(0, tf+.1, 1./30.)
dt = 0.001
tt = np.arange(0, tf+dt, dt)
tt = np.concatenate( [tt, snaptimes])
tt = np.unique(tt)

currentT = 0.


partitions = lp.sort3D(Mx, My, Mz)

for i, t in enumerate(tt[:-1]):
    dt = tt[i+1]-tt[i]
    if tt[i]+dt>tf:
        dt = tf-tt[i] 
    currentT += dt
    
    step1(eg, lp)
    if i == 0:
        print(np.mean(lp.M[:,1]))
        step2(eg, lp)
        print(np.mean(lp.M[:,1]))
    step3(eg, lp)
    step4(eg, lp)
    step5(eg, lp, ls)
    step7(eg, lp)
    step8(eg, lp)
    step10(eg, lp)
        
    
    if tt[i] in snaptimes:
        fig, ax = createPlot()
        for k, part in enumerate(partitions):
            plt.plot(lp.X[part,0], lp.X[part,1], shapes[ floor(k/5) ], c=colors[k%5], ms=2)
        v = np.linspace(0., 800, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], np.sum(eg.mm/eg.hx/eg.hy/eg.hz, axis=-1),
                        levels=v, extend='both')
        ax.contour( eg.xx[:,:,0], eg.yy[:,:,0], ls.phi[:,:,0], [0.],
                          colors='k' )
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case4/particles'+str(i)+'.png')
        plt.show()
        plt.close()
        
        fig, ax = createPlot()
        v = np.linspace(-1., 1, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], eg.uu[:,:,Nx//2],
                        levels=v, extend='both')
        ax.contour( eg.xx[:,:,0], eg.yy[:,:,0], ls.phi[:,:,0], [0.],
                          colors='k' )
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case4/U'+str(i)+'.png')
        plt.close()
        
        fig, ax = createPlot()
        v = np.linspace(-1., 1, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], eg.vv[:,:,Nx//2],
                        levels=v, extend='both')
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case4/V'+str(i)+'.png')
        plt.close()
        
        lp.saveTxt('case4/', i)
        
        





# Case 5

In [ ]:
# %%prun -s cumulative
## parameters
g = 9.81
μ = .03 ## coeff of friction

# material properties
E0 = 1.4e5 ## initial young modulus
ρ0 = 4e2 ## initial density & currently unused

ν = .2 ## poisson ration
λ0 = E0*ν/(1.+ν)/(1.-2*ν)
μ0 = E0/(1.+ν)/2

ξ = 5 ## Hardening coefficient
θc = 2.5e-2 ## Critical compression
θs = 7.5e-3 ## Critical stretch






eg = eulerianGrid(Nx, Ny, Nz)
lp = lagrangianParticles(numberParticles)
ls = levelSet(Nx, Ny, Nz)

snaptimes = np.arange(0, tf+.1, 1./60.)
dt = 0.001
tt = np.arange(0, tf+dt, dt)
tt = np.concatenate( [tt, snaptimes])
tt = np.unique(tt)

currentT = 0.


partitions = lp.sort3D(Mx, My, Mz)

for i, t in enumerate(tt[:-1]):
    dt = tt[i+1]-tt[i]
    if tt[i]+dt>tf:
        dt = tf-tt[i] 
    currentT += dt
    
    step1(eg, lp)
    if i == 0:
        print(np.mean(lp.M[:,1]))
        step2(eg, lp)
        print(np.mean(lp.M[:,1]))
    step3(eg, lp)
    step4(eg, lp)
    step5(eg, lp, ls)
    step7(eg, lp)
    step8(eg, lp)
    step10(eg, lp)
        
    
    if tt[i] in snaptimes:
        fig, ax = createPlot()
        for k, part in enumerate(partitions):
            plt.plot(lp.X[part,0], lp.X[part,1], shapes[ floor(k/5) ], c=colors[k%5], ms=2)
        v = np.linspace(0., 800, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], np.sum(eg.mm/eg.hx/eg.hy/eg.hz, axis=-1),
                        levels=v, extend='both')
        ax.contour( eg.xx[:,:,0], eg.yy[:,:,0], ls.phi[:,:,0], [0.],
                          colors='k' )
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case5/particles'+str(i)+'.png')
        plt.show()
        plt.close()
        
        fig, ax = createPlot()
        v = np.linspace(-1., 1, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], eg.uu[:,:,Nx//2],
                        levels=v, extend='both')
        ax.contour( eg.xx[:,:,0], eg.yy[:,:,0], ls.phi[:,:,0], [0.],
                          colors='k' )
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case5/U'+str(i)+'.png')
        plt.close()
        
        fig, ax = createPlot()
        v = np.linspace(-1., 1, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], eg.vv[:,:,Nx//2],
                        levels=v, extend='both')
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case5/V'+str(i)+'.png')
        plt.close()
        
        lp.saveTxt('case5/', i)
        
        





# Case 6

In [ ]:
%run lagrangianParticles.ipynb
# %%prun -s cumulative
## parameters
g = 9.81
μ = .03 ## coeff of friction

# material properties
E0 = 4.8e4 ## initial young modulus
ρ0 = 4e2 ## initial density & currently unused

ν = .2 ## poisson ration
λ0 = E0*ν/(1.+ν)/(1.-2*ν)
μ0 = E0/(1.+ν)/2

ξ = 10 ## Hardening coefficient
θc = 2.5e-2 ## Critical compression
θs = 7.5e-3 ## Critical stretch






eg = eulerianGrid(Nx, Ny, Nz)
lp = lagrangianParticles(numberParticles)
ls = levelSet(Nx, Ny, Nz)

snaptimes = np.arange(0, tf+.1, 1./30.)
dt = 0.001
tt = np.arange(0, tf+dt, dt)
tt = np.concatenate( [tt, snaptimes])
tt = np.unique(tt)

currentT = 0.


partitions = lp.sort3D(Mx, My, Mz)

for i, t in enumerate(tt[:-1]):
        
    print(i)
    dt = tt[i+1]-tt[i]
    if tt[i]+dt>tf:
        dt = tf-tt[i] 
    currentT += dt
    
    step1(eg, lp)
    if i == 0:
        print(np.mean(lp.M[:,1]))
        step2(eg, lp)
        print(np.mean(lp.M[:,1]))
    step3(eg, lp)
    step4(eg, lp)
    step5(eg, lp, ls)
    step7(eg, lp)
    step8(eg, lp)
    step10(eg, lp)
        
    
    if tt[i] in snaptimes:
        fig, ax = createPlot()
        for k, part in enumerate(partitions):
            plt.plot(lp.X[part,0], lp.X[part,1], shapes[ floor(k/5) ], c=colors[k%5], ms=2)
        v = np.linspace(0., 800, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], np.sum(eg.mm/eg.hx/eg.hy/eg.hz, axis=-1),
                        levels=v, extend='both')
        ax.contour( eg.xx[:,:,0], eg.yy[:,:,0], ls.phi[:,:,0], [0.],
                          colors='k' )
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case6/particles'+str(i)+'.png')
        plt.show()
        plt.close()
        
        fig, ax = createPlot()
        v = np.linspace(-1., 1, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], eg.uu[:,:,Nx//2],
                        levels=v, extend='both')
        ax.contour( eg.xx[:,:,0], eg.yy[:,:,0], ls.phi[:,:,0], [0.],
                          colors='k' )
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case6/U'+str(i)+'.png')
        plt.close()
        
        fig, ax = createPlot()
        v = np.linspace(-1., 1, 41, endpoint=True)
        cs = ax.contourf( eg.xx[:,:,0], eg.yy[:,:,0], eg.vv[:,:,Nx//2],
                        levels=v, extend='both')
        fig.colorbar(cs, orientation='vertical', ticks=v[::4], shrink=0.999, extendrect=True)
        plt.savefig('case6/V'+str(i)+'.png')
        plt.close()
        
        lp.saveTxt('case6/', i)
        
        



